In [6]:
import cv2
import numpy as np
import math


In [7]:
def Hough(image):
    dst = cv2.Canny(image, 50, 200, None, 3)

    val = 250

    cdst = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    lines = cv2.HoughLines(dst, 1, np.pi / 180, val, None, 0, 0)

    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
            pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
            cv2.line(cdst, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)
    return cdst

In [8]:
video = cv2.VideoCapture("res/vid1.mov")

fourcc = cv2.VideoWriter_fourcc(*"XVID")
output = cv2.VideoWriter("out/hough.avi", fourcc, 25.0, (1920, 1080))

while True:
    ret, frame = video.read()
    if not ret:
        break
    # plt.imshow(frame)

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.Canny(frame, 50, 200, None, 3)
    frame = Hough(frame)
    output.write(frame)

output.release()

In [9]:
video = cv2.VideoCapture("res/vid2.mov")
frames = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    frames.append(frame)


In [17]:
output = cv2.VideoWriter("out/diff.avi", fourcc, 25.0, (1280, 720))
masks = []
for i in range(0, len(frames) - 1):
    frame1 = cv2.resize(frames[i], (1280, 720))
    frame2 = cv2.resize(frames[i + 1], (1280, 720))
    difference = cv2.subtract(frame1, frame2)
    #
    Conv_hsv_Gray = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(Conv_hsv_Gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    difference[mask != 255] = [0, 0, 255]
    # output.write(cv2.bitwise_and(frame1, frame1, mask=mask))
    #
    output.write(difference)

output.release()